In [ ]:
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
from time import sleep
from PIL import Image
from selenium import webdriver
from IPython.display import clear_output

In [ ]:
# Initializations
list_num = [i for i in map(int, open("../data/images/idxdata.csv", "r").read().split(","))]
label_names = [
    'Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney',
    'Crosswalk', 'Hydrant', 'Motorcycle', 'Other', 'Palm', 'Stair',
    'Traffic Light'
]

def save_progress():
    open("../data/images/idxdata.csv", "w").write(",".join(map(str, list_num)))

In [ ]:
# WEB DRIVER FUNCTIONS
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get("https://patrickhlauke.github.io/recaptcha/")
driver.fullscreen_window()

driver.switch_to.default_content()
driver.find_element_by_tag_name("iframe").click()
sleep(0.33)

# Find the right iframe in case of multiple redundant tags
iframes = [i for i in driver.find_elements_by_tag_name("iframe")]
correctIFrame = None
for i in iframes:
    driver.switch_to.frame(i)
    buttons = driver.find_elements_by_tag_name("button")
    if len(driver.find_elements_by_tag_name("button")) != 0:
        print("correct iframe found")
        correctIFrame = i
    driver.switch_to.default_content()    
driver.switch_to.frame(correctIFrame)

def reload():
    driver.find_element_by_tag_name("button").click()

def check_table():
    table_elements = []
    rows = driver.find_elements_by_tag_name("tr")
    for i in rows:
        columns = i.find_elements_by_tag_name("td")
        for j in columns:
            table_elements.append(j)
    return table_elements  

def img_element_with_id(id):
    images = driver.find_elements_by_tag_name("img")
    for i in images:
        if i.get_attribute("class") == "rc-image-tile-33":
            return i

# Get the correct captcha size and type (3x3)
def find_3x3_captcha():
    img_element = img_element_with_id("rc-image-tile-33")
    while img_element is None:
        reload()
        sleep(1)
        img_element = img_element_with_id("rc-image-tile-33")
    return check_table()

In [ ]:
# IMAGE FUNCTIONS
def download_image():
    urllib.request.urlretrieve(driver.find_element_by_class_name("rc-image-tile-33").get_attribute("src"), "../data/solver/captcha.jpeg")
    return Image.open("../data/solver/captcha.jpeg")

def show_image(img):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(np.array(img))
    plt.show()

def process_image(img):
    if img.mode == "RGBA":
        img = img.convert("RGB")
    img = img.resize((100, 100))
    return img

def crop_image(image):
    width, height = image.size
    grid_width = width // 3
    grid_height = height // 3
    cropped_images = []
    for i in range(3):
        for j in range(3):
            left = j * grid_width
            upper = i * grid_height
            right = (j + 1) * grid_width
            lower = (i + 1) * grid_height
            cropped = image.crop((left, upper, right, lower))
            cropped_images.append(process_image(cropped))
    return cropped_images

In [ ]:
def validate_input():
    lower_case_labels = [i for i in map(str.lower, label_names)]
    lower_case_labels[11] = "traffic_light"
    input_valid = False
    while input_valid == False:
        print("VALID INPUTS: %s" % ", ".join(lower_case_labels))
        user_input = input(">").split(" ")
        for i in user_input:
            if i in lower_case_labels:
                input_valid = True
            else:
                input_valid = False
                break
    return [lower_case_labels.index(i) for i in user_input]

def save_images(indices, image):
    for i in indices:
        image.save("../data/images/%s/%d.jpeg" % (label_names[i], list_num[i]))
        list_num[i] += 1



In [ ]:
def main():
    while True:
        sleep(2)
        find_3x3_captcha()
        images = crop_image(download_image())
        index = 0
        for i in images:
            clear_output()
            sleep(0.2)
            show_image(i)
            label_indices = validate_input()
            save_images(label_indices, i)
        clear_output()
        user_input = input("press enter to continue or type exit to quit")
        if user_input == "exit":
            break
    save_progress()
    print("Progress saved to idxdata.csv")

In [ ]:
main()